In [ ]:
import tensorflow as tf
print("✅ TensorFlow version:", tf.__version__)
print("🚀 GPU available:", tf.config.list_physical_devices('GPU'))


✅ TensorFlow version: 2.19.0
🚀 GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

BASE_DIR = "/content/drive/MyDrive/panda_data/Dataset"

# -----------------------
# Train / Test splits
# -----------------------
TRAIN_NORMAL = ["Yaas", "Amphan", "Gulab", "Hikka"]
TEST_NORMAL  = ["Burevi", "Mekunu"]

ALL_NON_CURVING = TRAIN_NORMAL + TEST_NORMAL

IMG_SIZE = (224, 224)

def collect_cyclone_images(base_dir, name_list):
    result = {}
    for folder in os.listdir(base_dir):
        for name in name_list:
            if name.lower() in folder.lower():
                full = os.path.join(base_dir, folder)
                imgs = sorted([
                    os.path.join(full, f)
                    for f in os.listdir(full)
                    if f.lower().endswith((".jpg", ".jpeg", ".png"))
                ])
                result[name] = imgs
    return result

# Train normals: 4 cyclones
train_normal_paths = collect_cyclone_images(BASE_DIR, TRAIN_NORMAL)

# Test normals: 2 cyclones
test_normal_paths = collect_cyclone_images(BASE_DIR, TEST_NORMAL)

# Curving cyclones: all folders that are NOT in non-curving list
curving_paths = {}
for folder in os.listdir(BASE_DIR):
    if not any(nc.lower() in folder.lower() for nc in ALL_NON_CURVING):
        full = os.path.join(BASE_DIR, folder)
        if not os.path.isdir(full):
            continue
        imgs = sorted([
            os.path.join(full, f)
            for f in os.listdir(full)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ])
        if len(imgs) > 0:
            curving_paths[folder] = imgs

print("Train normals:", {k: len(v) for k,v in train_normal_paths.items()})
print("Test  normals:", {k: len(v) for k,v in test_normal_paths.items()})
print("Curving cyclones:", {k: len(v) for k,v in curving_paths.items()})

def load_image(path):
    img = load_img(path, target_size=IMG_SIZE)
    arr = img_to_array(img).astype("float32") / 255.0
    return arr


Train normals: {'Yaas': 929, 'Amphan': 913, 'Gulab': 483, 'Hikka': 241}
Test  normals: {'Burevi': 850, 'Mekunu': 217}
Curving cyclones: {'Oct21_070410 (Nivar)': 788, 'Oct21_70412 (Fani)': 1646, 'Oct21_070405 (Nisarga)': 807, 'Oct21_070311 (Tauktae)': 923, 'Jan22_072151 (Maha)': 1163, 'Jan22_072152 (Bulbul)': 675, 'Jan22_072150 (Kyaar)': 1358, 'Jan22_072149 (Jawad)': 374, 'Apr22_074339 (Titli)': 1070, 'Dec21_071404 (Vayu)': 1561, 'Apr22_074174 (Daye)': 92, 'Apr22_074172 (Phethai)': 206, 'Apr22_074108 (Shaheen)': 925, 'Apr22_074107 (Gati)': 467}


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50

IMG_SIZE = (224, 224)
FEATURE_DIM = 2048
FINE_TUNE_AT = 140
RESNET_WT_PATH = "/content/drive/MyDrive/panda_data/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

def build_panda_model(pretrained_path, fine_tune_at=FINE_TUNE_AT,
                      feature_dim=FEATURE_DIM):
    """
    Method-B: Exact PANDA style (single-frame, 3-channel input).
    """
    inp = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), name="panda_input")

    # ResNet50 backbone (ImageNet)
    base = ResNet50(include_top=False,
                    weights=None,
                    input_tensor=inp,
                    pooling="avg",
                    name="resnet50_backbone")

    # Load ImageNet weights
    base.load_weights(pretrained_path, by_name=True, skip_mismatch=True)
    print("✓ Loaded ResNet50 ImageNet weights")

    # Freeze early layers, fine-tune later layers
    for layer in base.layers[:fine_tune_at]:
        layer.trainable = False
    for layer in base.layers[fine_tune_at:]:
        layer.trainable = True

    # Projection head + L2 norm (PANDA style)
    x = layers.Dense(feature_dim, activation=None, name="projection_dense")(base.output)
    x = layers.Lambda(lambda z: tf.math.l2_normalize(z, axis=1), name="l2_norm")(x)

    model = Model(inputs=inp, outputs=x, name="PANDA_MethodB")
    return model

model = build_panda_model(RESNET_WT_PATH)
model.summary()


✓ Loaded ResNet50 ImageNet weights


Model: "PANDA_MethodB"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ panda_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ panda_input[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 27,784,064 (105.99 MB)

 Trainable params: 19,174,400 (73.14 MB)

 Non-trainable params: 8,609,664 (32.84 MB)

In [ ]:
import numpy as np

def train_generator_single(img_list, batch_size=32, shuffle=True):
    idxs = np.arange(len(img_list))
    while True:
        if shuffle:
            np.random.shuffle(idxs)
        for i in range(0, len(idxs), batch_size):
            sub = idxs[i:i+batch_size]
            batch = [load_image(img_list[j]) for j in sub]
            yield np.stack(batch, axis=0)

# Flatten all train-normal images
train_img_list = []
for name, imgs in train_normal_paths.items():
    train_img_list.extend(imgs)
print("Total train-normal images:", len(train_img_list))

def fisher_proxy_loss(model, batch):
    with tf.GradientTape() as tape:
        feats = model(batch, training=True)
        proxy = tf.reduce_sum(tf.square(feats))
    grads = tape.gradient(proxy, model.trainable_weights)
    return grads

def compute_fisher(model, train_img_list, batch_size=16, num_batches=40):
    fisher_diag = [np.zeros_like(w.numpy(), dtype=np.float32)
                   for w in model.trainable_weights]

    gen = train_generator_single(train_img_list, batch_size=batch_size, shuffle=True)

    print(f"Running Fisher: {num_batches} batches × {batch_size}")
    for b in range(num_batches):
        batch = next(gen)
        grads = fisher_proxy_loss(model, batch)

        for i, g in enumerate(grads):
            if g is not None:
                fisher_diag[i] += (g.numpy() ** 2)

        if (b+1) % 5 == 0 or (b+1) == num_batches:
            print(f"  processed {b+1}/{num_batches}")

    for i in range(len(fisher_diag)):
        fisher_diag[i] /= float(num_batches)

    print("✓ Fisher computation complete")
    return fisher_diag

fisher_diag = compute_fisher(model, train_img_list, batch_size=16, num_batches=40)
theta0 = [w.numpy().copy() for w in model.trainable_weights]


Total train-normal images: 2566
Running Fisher: 40 batches × 16
  processed 5/40
  processed 10/40
  processed 15/40
  processed 20/40
  processed 25/40
  processed 30/40
  processed 35/40
  processed 40/40
✓ Fisher computation complete


In [ ]:
import math

BATCH_SIZE = 64

def extract_features_for_list(model, paths, batch_size=64):
    n = len(paths)
    feats = np.zeros((n, model.output_shape[1]), dtype=np.float32)
    i = 0
    while i < n:
        j = min(i + batch_size, n)
        batch_imgs = np.stack([load_image(p) for p in paths[i:j]], axis=0)
        batch_feats = model(batch_imgs, training=False).numpy()
        feats[i:j] = batch_feats
        i = j
    return feats

print("Extracting embeddings for centroid (pre-adaptation)...")
train_feats_pre = extract_features_for_list(model, train_img_list, batch_size=BATCH_SIZE)
centroid = train_feats_pre.mean(axis=0).astype(np.float32)
print("Centroid shape:", centroid.shape)


Extracting embeddings for centroid (pre-adaptation)...
Centroid shape: (2048,)


In [ ]:
import tensorflow as tf

EPOCHS = 3
LR = 1e-4
LAMBDA_EWC = 1e4    # EWC strength (as in PANDA spirit)
BATCH_SIZE_TRAIN = 32

optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
centroid_tf = tf.constant(centroid)

# Turn fisher to tensors
fisher_tf = [tf.constant(f, dtype=tf.float32) for f in fisher_diag]
theta0_tf = [tf.constant(w0, dtype=tf.float32) for w0 in theta0]

gen_train = train_generator_single(train_img_list, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
steps_per_epoch = math.ceil(len(train_img_list) / BATCH_SIZE_TRAIN)

print(f"Starting PANDA-EWC training (Method-B): {EPOCHS} epochs, {steps_per_epoch} steps/epoch")

for epoch in range(1, EPOCHS + 1):
    epoch_loss = 0.0
    epoch_comp = 0.0
    epoch_ewc = 0.0

    for step in range(steps_per_epoch):
        batch = next(gen_train)
        batch_tf = tf.convert_to_tensor(batch, dtype=tf.float32)

        with tf.GradientTape() as tape:
            embeds = model(batch_tf, training=True)

            # Compactness loss ||f(x) - c||^2
            diff = embeds - centroid_tf
            comp_loss_per = tf.reduce_sum(tf.square(diff), axis=1)
            compactness_loss = tf.reduce_mean(comp_loss_per)

            # EWC penalty
            ewc_penalty = 0.0
            for w, w0, F in zip(model.trainable_weights, theta0_tf, fisher_tf):
                w_diff = w - w0
                ewc_penalty += tf.reduce_sum(F * tf.square(w_diff))
            ewc_penalty = 0.5 * LAMBDA_EWC * ewc_penalty

            loss = compactness_loss + ewc_penalty

        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        epoch_loss += float(loss.numpy())
        epoch_comp += float(compactness_loss.numpy())
        epoch_ewc += float(ewc_penalty.numpy())

        if (step+1) % 20 == 0 or (step+1) == steps_per_epoch:
            print(f"Epoch {epoch}/{EPOCHS} step {step+1}/{steps_per_epoch} "
                  f"| loss={epoch_loss/(step+1):.6e} "
                  f"| comp={epoch_comp/(step+1):.6e} "
                  f"| ewc={epoch_ewc/(step+1):.6e}")

    print(f"--- End Epoch {epoch} | avg_loss={epoch_loss/steps_per_epoch:.6e} "
          f"| comp={epoch_comp/steps_per_epoch:.6e} "
          f"| ewc={epoch_ewc/steps_per_epoch:.6e}")

# Save adapted weights (Method-B)
METHOD_B_WEIGHTS = "/content/drive/MyDrive/panda_data/panda_methodB_adapted.weights.h5"
model.save_weights(METHOD_B_WEIGHTS)
print("✓ Saved Method-B adapted weights:", METHOD_B_WEIGHTS)


Starting PANDA-EWC training (Method-B): 3 epochs, 81 steps/epoch
Epoch 1/3 step 20/81 | loss=2.728742e-01 | comp=2.728742e-01 | ewc=3.265171e-13
Epoch 1/3 step 40/81 | loss=1.399987e-01 | comp=1.399987e-01 | ewc=5.837882e-13
Epoch 1/3 step 60/81 | loss=9.406276e-02 | comp=9.406276e-02 | ewc=7.156802e-13
Epoch 1/3 step 80/81 | loss=7.090226e-02 | comp=7.090226e-02 | ewc=7.907391e-13
Epoch 1/3 step 81/81 | loss=7.004293e-02 | comp=7.004293e-02 | ewc=7.936554e-13
--- End Epoch 1 | avg_loss=7.004293e-02 | comp=7.004293e-02 | ewc=7.936554e-13
Epoch 2/3 step 20/81 | loss=1.167000e-03 | comp=1.167000e-03 | ewc=1.036266e-12
Epoch 2/3 step 40/81 | loss=1.093625e-03 | comp=1.093625e-03 | ewc=1.044681e-12
Epoch 2/3 step 60/81 | loss=1.031330e-03 | comp=1.031330e-03 | ewc=1.052941e-12
Epoch 2/3 step 80/81 | loss=9.789206e-04 | comp=9.789206e-04 | ewc=1.061165e-12
Epoch 2/3 step 81/81 | loss=9.748501e-04 | comp=9.748501e-04 | ewc=1.061576e-12
--- End Epoch 2 | avg_loss=9.748501e-04 | comp=9.748501e

In [ ]:
# Reload model (optional, for clean state)
model = build_panda_model(RESNET_WT_PATH)
model.load_weights(METHOD_B_WEIGHTS)
print("✓ Loaded Method-B adapted weights")

# Train embeddings (normal reference)
train_feats = extract_features_for_list(model, train_img_list, batch_size=BATCH_SIZE)
print("Train feats shape:", train_feats.shape)

# Test-normal embeddings (per cyclone)
test_normal_feats = {}
for name, imgs in test_normal_paths.items():
    feats = extract_features_for_list(model, imgs, batch_size=BATCH_SIZE)
    test_normal_feats[name] = feats
    print(f"{name} test feats shape:", feats.shape)

# Curving embeddings (per cyclone)
curving_feats = {}
for name, imgs in curving_paths.items():
    feats = extract_features_for_list(model, imgs, batch_size=BATCH_SIZE)
    curving_feats[name] = feats
    print(f"{name} curving feats shape:", feats.shape)


✓ Loaded ResNet50 ImageNet weights
✓ Loaded Method-B adapted weights
Train feats shape: (2566, 2048)
Burevi test feats shape: (850, 2048)
Mekunu test feats shape: (217, 2048)
Oct21_070410 (Nivar) curving feats shape: (788, 2048)
Oct21_70412 (Fani) curving feats shape: (1646, 2048)
Oct21_070405 (Nisarga) curving feats shape: (807, 2048)
Oct21_070311 (Tauktae) curving feats shape: (923, 2048)
Jan22_072151 (Maha) curving feats shape: (1163, 2048)
Jan22_072152 (Bulbul) curving feats shape: (675, 2048)
Jan22_072150 (Kyaar) curving feats shape: (1358, 2048)
Jan22_072149 (Jawad) curving feats shape: (374, 2048)
Apr22_074339 (Titli) curving feats shape: (1070, 2048)
Dec21_071404 (Vayu) curving feats shape: (1561, 2048)
Apr22_074174 (Daye) curving feats shape: (92, 2048)
Apr22_074172 (Phethai) curving feats shape: (206, 2048)
Apr22_074108 (Shaheen) curving feats shape: (925, 2048)
Apr22_074107 (Gati) curving feats shape: (467, 2048)


In [ ]:
from sklearn.neighbors import NearestNeighbors

K = 2  # PANDA typically uses small k for kNN scoring

# Fit kNN on train-normal embeddings
knn = NearestNeighbors(n_neighbors=K, metric="euclidean")
knn.fit(train_feats)

def knn_scores(feats, knn_model, k=K):
    dists, _ = knn_model.kneighbors(feats, n_neighbors=k)
    # PANDA: average distance to k nearest neighbors as anomaly score
    return dists.mean(axis=1)

# Example: compute scores for one normal and one curving cyclone
for name, feats in test_normal_feats.items():
    scores = knn_scores(feats, knn)
    print(f"[Method-B] Test-normal {name}: score stats: min={scores.min():.4f}, "
          f"mean={scores.mean():.4f}, max={scores.max():.4f}")

for name, feats in curving_feats.items():
    scores = knn_scores(feats, knn)
    print(f"[Method-B] Curving {name}: score stats: min={scores.min():.4f}, "
          f"mean={scores.mean():.4f}, max={scores.max():.4f}")


[Method-B] Test-normal Burevi: score stats: min=0.0238, mean=0.0300, max=0.0385
[Method-B] Test-normal Mekunu: score stats: min=0.0239, mean=0.0336, max=0.0532
[Method-B] Curving Oct21_070410 (Nivar): score stats: min=0.0264, mean=0.0325, max=0.0542
[Method-B] Curving Oct21_70412 (Fani): score stats: min=0.0221, mean=0.0303, max=0.0452
[Method-B] Curving Oct21_070405 (Nisarga): score stats: min=0.0248, mean=0.0298, max=0.0464
[Method-B] Curving Oct21_070311 (Tauktae): score stats: min=0.0248, mean=0.0306, max=0.0445
[Method-B] Curving Jan22_072151 (Maha): score stats: min=0.0258, mean=0.0363, max=0.0665
[Method-B] Curving Jan22_072152 (Bulbul): score stats: min=0.0253, mean=0.0354, max=0.0650
[Method-B] Curving Jan22_072150 (Kyaar): score stats: min=0.0239, mean=0.0359, max=0.0566
[Method-B] Curving Jan22_072149 (Jawad): score stats: min=0.0264, mean=0.0344, max=0.0473
[Method-B] Curving Apr22_074339 (Titli): score stats: min=0.0232, mean=0.0299, max=0.0433
[Method-B] Curving Dec21_071

In [ ]:
# ==========================================
# METHOD-B — PART-8 (Final Clean Version)
# ==========================================
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img

SAVE_PLOT_DIR = "/content/panda_run/methodB_plots"
os.makedirs(SAVE_PLOT_DIR, exist_ok=True)

def compute_scores_and_plot(title, feats, img_paths):
    # Compute kNN scores using previously built KNN model
    scores = knn_scores(feats, knn)

    # Curving index = where anomaly peaks
    idx = int(np.argmax(scores))
    max_score = scores[idx]

    print(f"[Method-B] {title}: Curving index = {idx}, Max score = {max_score:.4f}")

    # Save scores for future analysis
    np.save(os.path.join(SAVE_PLOT_DIR, f"{title}_scores.npy"), scores)

    # === Plot anomaly curve ===
    plt.figure(figsize=(14,4))
    plt.plot(scores, linewidth=2)
    plt.axvline(idx, color='red', linestyle='--', label=f"Peak @ {idx}")
    plt.title(f"{title} — PANDA Anomaly Score Curve")
    plt.xlabel("Frame index")
    plt.ylabel("kNN Distance (Anomaly Score)")
    plt.legend()
    plt.grid(True)

    plot_path = os.path.join(SAVE_PLOT_DIR, f"{title}_plot.png")
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.close()
    print("✔ Plot saved:", plot_path)

    # === Save 3 deviation frames ===
    frame_save_dir = os.path.join(SAVE_PLOT_DIR, f"{title}_frames")
    os.makedirs(frame_save_dir, exist_ok=True)

    for k in range(3):
        if idx + k < len(img_paths):
            img = load_img(img_paths[idx + k])
            img_file = os.path.join(frame_save_dir, f"frame_{idx+k}.png")
            img.save(img_file)

    print("✔ Deviation frames saved:", frame_save_dir)
    print("----------------------------------------------------\n")

    return scores


# ==========================================
# RUN FOR ALL CYCLONES
# ==========================================

# TRAIN NORMAL CYCLONES
print("\n=== TRAIN NORMAL CYCLONES ===\n")
for name, img_paths in train_normal_paths.items():
    feats = extract_features_for_list(model, img_paths)
    compute_scores_and_plot(f"TRAIN_{name}", feats, img_paths)

# TEST NORMAL CYCLONES
print("\n=== TEST NORMAL CYCLONES ===\n")
for name, img_paths in test_normal_paths.items():
    feats = extract_features_for_list(model, img_paths)
    compute_scores_and_plot(f"TEST_{name}", feats, img_paths)

# CURVING CYCLONES
print("\n=== CURVING CYCLONES ===\n")
for name, img_paths in curving_paths.items():
    feats = extract_features_for_list(model, img_paths)
    compute_scores_and_plot(f"CURVING_{name}", feats, img_paths)

print("\n🎉 All Method-B Plots and Frames Saved Successfully!")
print("Results saved in:", SAVE_PLOT_DIR)



=== TRAIN NORMAL CYCLONES ===

[Method-B] TRAIN_Yaas: Curving index = 185, Max score = 0.0104
✔ Plot saved: /content/panda_run/methodB_plots/TRAIN_Yaas_plot.png
✔ Deviation frames saved: /content/panda_run/methodB_plots/TRAIN_Yaas_frames
----------------------------------------------------

[Method-B] TRAIN_Amphan: Curving index = 827, Max score = 0.0121
✔ Plot saved: /content/panda_run/methodB_plots/TRAIN_Amphan_plot.png
✔ Deviation frames saved: /content/panda_run/methodB_plots/TRAIN_Amphan_frames
----------------------------------------------------

[Method-B] TRAIN_Gulab: Curving index = 115, Max score = 0.0080
✔ Plot saved: /content/panda_run/methodB_plots/TRAIN_Gulab_plot.png
✔ Deviation frames saved: /content/panda_run/methodB_plots/TRAIN_Gulab_frames
----------------------------------------------------

[Method-B] TRAIN_Hikka: Curving index = 238, Max score = 0.0096
✔ Plot saved: /content/panda_run/methodB_plots/TRAIN_Hikka_plot.png
✔ Deviation frames saved: /content/panda_run

In [ ]:
!mkdir -p "/content/drive/MyDrive/panda_data/panda_run/methodB_plots"
!cp -r /content/panda_run/methodB_plots/* "/content/drive/MyDrive/panda_data/panda_run/methodB_plots/"
